In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib

In [ ]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from qgsw.utils.sorting import sort_files
from qgsw.run_summary import RunSummary

In [ ]:
ROOT = Path(os.path.abspath('')).parent

In [ ]:
folder = ROOT.joinpath("output/g5k/two_layers_baroclinic_100km")
run = RunSummary.from_file(folder.joinpath("_summary.toml"))

In [ ]:
_, files = sort_files(folder.rglob("*.npz"), prefix=run.configuration.model.prefix, suffix = ".npz")

In [ ]:
plt.ion()
cmap = plt.cm.bwr
fig, axes = plt.subplots(1,3, figsize=(18,6), dpi=150, layout='tight')
for file in files[4:]:
    omega = np.load(file)["omega"]
    cbar = axes[0].imshow(omega[0,0], cmap=cmap)
    cbar = axes[1].imshow(omega[0,1], cmap=cmap)
    mask_omega = np.logical_or(np.abs(omega[0,0]) > 1e-2, np.abs(omega[0,1]) > 1e-2)
    ratio = np.zeros(mask_omega.shape)
    ratio[mask_omega] = omega[0,1][mask_omega]/omega[0,0][mask_omega]
    ratio[np.abs(ratio) > np.quantile(np.abs(ratio[np.abs(ratio) > 1e-10]),0.01) ] = 0
    vmax = np.abs(ratio).max()
    cbar = axes[2].imshow(omega[0,0]/omega[0,1], cmap=cmap, vmin = -vmax, vmax=vmax)
    plt.pause(0.05)

In [ ]:
file = files[50]
omega = np.load(file)["omega"]
quantile_value = 0.75

In [ ]:
%matplotlib inline
top = omega[0,0]
abs_top = np.abs(top)[np.abs(top) > 1e-5]
quantile = np.quantile(abs_top, quantile_value)
top_below_quantile = np.abs(top) > quantile

fig, axes = plt.subplots(1, 2, figsize=(12,6))
fig.suptitle("Surface")

cbar = axes[0].imshow(top)
fig.colorbar(cbar, ax=axes[0])
axes[0].set_title("Vorticity")

cbar = axes[1].imshow(top_below_quantile)
fig.colorbar(cbar, ax=axes[1])
axes[1].set_title(f"Vorticity above {quantile_value} quantile")

plt.show()

In [ ]:
%matplotlib inline
bottom = omega[0,1]
abs_bottom = np.abs(bottom)[np.abs(bottom) > 1e-5]
quantile = np.quantile(abs_bottom, quantile_value)
bottom_below_quantile = np.abs(bottom) > quantile

fig, axes = plt.subplots(1, 2, figsize=(12,6))
fig.suptitle("Bottom")

cbar = axes[0].imshow(bottom)
fig.colorbar(cbar, ax=axes[0])
axes[0].set_title("Vorticity")

cbar = axes[1].imshow(bottom_below_quantile)
fig.colorbar(cbar, ax=axes[1])
axes[1].set_title(f"Vorticity above {quantile_value} quantile")

plt.show()

In [ ]:
ratio = np.zeros(top.shape)
below_quantiles = top_below_quantile & bottom_below_quantile
ratio[below_quantiles] = bottom[below_quantiles] / top[below_quantiles]
np.mean(ratio[below_quantiles] )

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(6,6))
fig.suptitle("Ratio")

cbar = axes.imshow(ratio, vmin = -np.max(np.abs(ratio)), vmax = np.max(np.abs(ratio)))
fig.colorbar(cbar, ax=axes)
axes.set_title("Vorticity Ratio")

plt.show()